In [1]:
import sys
import os
import time
from datetime import datetime

sys.path.append("..")
# 自分のpath
os.chdir("/home/nakamura/HAI/main_server/hai")

import pymongo
from PIL import Image
from matplotlib import pylab as plt
from matplotlib.animation import ArtistAnimation
import matplotlib.animation as animation
import numpy as np
from bson.objectid import ObjectId
from sklearn.decomposition import PCA

from _app import app
from database import mongo
from notebooks.utils.utils import visualize, display_latest_image, display_image, print_time, strtime_to_epoch, epoch_to_strtime, pair_images

In [2]:
app.config['PORT_DB']

20202

In [3]:
from controllers.learner.img2vec import NNFeatures

In [4]:
nn = NNFeatures()

In [5]:
prints = mongo.print.find({"user_name": "koki", 'time': {"$gt": 1509675690.059450}, 
                           "$or": [{"text": "play"}, {"text": 'study'}, {'text': 'sleep'}, {'text': 'stop'}]}, sort=[("_id", pymongo.ASCENDING)])
prints = list(prints)

In [6]:
st = prints[0]['time'] - 0.1
et = prints[-1]['time'] + 0.1

images0 = mongo.images.find({'time': {"$gt": st, "$lt": et}, "cam_id": "webcam0"}, sort=[("_id", pymongo.ASCENDING)])
# images1 = mongo.images.find({'time': {"$gt": st, "$lt": et}, "cam_id": "webcam1"}, sort=[("_id", pymongo.ASCENDING)])

In [7]:
data = pair_images(images0)

In [8]:
labels = []
for im0, im1 in data:
    for l in range(len(prints) - 1):
        if prints[l+1]['time'] >= im0['time']:
            labels.append(l)
            break

In [10]:
len(data)

363

In [12]:
input_image = []
for d in data:
    input_image.append(Image.open(app.config['RAW_IMG_DIR'] + d[0]['filename']))
for d in data:
    input_image.append(Image.open(app.config['RAW_IMG_DIR'] + d[1]['filename']))

In [14]:
len(input_image)

726

In [17]:
features = nn.img2vec(input_image)

0/726


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/THC/generic/THCStorage.cu:66